##### Pacotes necessários (_DelimitedFiles_ pode ser ignorado caso não queira exportar soluções em .txt)

In [ ]:
using JuMP, Cbc, DelimitedFiles

##### Objeto do modelo

In [ ]:
model = JuMP.Model(Cbc.Optimizer)
set_silent(model) # silenciar solver log

##### Parâmetros e conjuntos do problema

In [ ]:
m, n = (6, 6)
I = 1:m
J = 1:n
K = [1,2,3,4,5,6,7,8]
big_m = m * n * K[end]

##### Variáveis

 - $x_{ijk}$:  assume-se 1 se o valor $k$ é atribuído na posição da linha $i$ e coluna $j$ (Binary)
 - $y_{ij}$: valor atribuído na posição da linha $i$ e coluna $j$ (Integer)



In [ ]:
@variable(model, x[i ∈ I, j ∈ J, k ∈ K], Bin);
@variable(model, 1 <= y[i ∈ I, j ∈ J] <= 8, Int);

##### Restrições

Para obter-se uma matriz com a solução do problema. Essa restrição não é necessário no modelo, foi criada apenas obter-se a solução desejada pela própria modelagem em vez de criar um trecho para "traduzir" a variável binária 3D para uma grelha 2D

 - $\quad y_{ij} = \sum_{k \in K} x_{ijk} \quad \forall i \in I, j \in J \quad \quad (1)$


In [ ]:
@constraint(model, [i ∈ I, j ∈ J], y[i,j] == sum(x[i,j,k] * k for k ∈ K))

Para garantir que um único número será atribuído em cada posição da grelha
 
 - $\quad \quad \sum_{k \in K} x_{ijk} = 1 \quad \forall i \in I, j \in J \qquad \qquad (2)$

In [ ]:
@constraint(model, [i ∈ I, j ∈ J], sum(x[i,j,k] for k ∈ K) == 1)

Para garantir que se um valor $k$ foi atribuído numa dada posição $a_{ij}$, esse mesmo $k$ não pode ser repetido em nenhuma posição com distância menor ou igual a $k$ na grelha

 - $\sum_{(\hat{i},\hat{j}) \in \{(\hat{i},\hat{j}) \mid \hat{i} \in I, \hat{j} \in J, \, |\hat{i} - i| + |\hat{j} - j| \leq k\}} x_{\hat{i}\hat{j}k} \leq M(1 - x_{ijk}) + x_{ijk} \quad \forall i \in I, j \in J, k \in K$

In [ ]:
for i ∈ I
    for j ∈ J
        for k ∈ K
            arr = [[i_hat, j_hat] for i_hat ∈ I for j_hat ∈ J if abs(i-i_hat) + abs(j-j_hat) <= k]
            @constraint(model, sum(x[i_bar, j_bar, k] for (i_bar, j_bar) in zip([[a[1] for a in arr]...], [[a[2] for a in arr]...])) <= big_m * (1 - x[i,j,k])  + x[i,j,k])
        end
    end
end

##### Função objetivo

 - Não há um objetivo especificado na descrição problema. Este foi criado para que pudesse ser formulado como um problema de otimização.
 
 - Optou-se por minimizar o somatório dos números dispostos na grelha (poderia ser maximizar).

In [ ]:
@objective(model, Min, sum(y[i,j] for i ∈ I for j ∈ J))

##### Solução

In [ ]:
optimize!(model)
println("Valor da função objetivo: $(objective_value(model))")
println("Solução: $(value.(y) |> Array{Int64})")


##### Várias soluções (opcional)

 - É possível gerar diversas soluções para o problema especificado através de várias execuções do solver. A cada iteração no laço "while" é verificado se o valor da função objetivo é maior que o da execução anterior e uma nova restrição é adicionada ao modelo para limitar inferiormente o valor da função objetivo pelo valor obtido na última execução.
 
 - Para ativar esta implementação e salvar as soluções em arquivos .txt basta substituir `false` por `true` no `if` da primeira linha do trecho de código abaixo.

In [ ]:
if false
    aux = true
    previous_of = 0
    d = Dict()
    count = 0
    while aux
        optimize!(model)
        status = termination_status(model)
        
        if status != MOI.OPTIMAL
            break
        end
        
        count -= -1
        @show count
        of = objective_value(model)
        @show of
        sol = round.(value.(y)) |> Array{Int64}
        @show sol

        if of > previous_of
            @constraint(model, sum(y[i,j] for i ∈ I for j ∈ J) >= of + 1)

            writedlm("sol_obj_$(of).txt", sol, ' ')

            d["$(of)"] = sol
            previous_of = of
        else
            aux = false
        end
    end
end